# Introduction to Image Classification using Multiple Models

In this JupyterLab notebook, we implement and compare three distinct image classification models using different frameworks and architectures. We will use the PIL library for image manipulation, ImageAI for MobileNetV2 classification, TensorFlow for ResNet-50, and PyTorch for a custom Simple CNN model. 

## Libraries and Dependencies

- **PIL (Python Imaging Library)**: Used for opening, manipulating, and saving image files.
- **ImageAI**: Provides pre-trained deep learning models for image classification tasks. We are using MobileNetV2 here.
- **TensorFlow & Keras**: Used for implementing the ResNet-50 model.
- **NumPy**: For numerical operations, especially array manipulations.
- **PyTorch**: A machine learning library used for our custom Simple CNN model.
- **torchvision.transforms**: Provides standard transformations for image preprocessing.
- **json**: To read JSON files containing class names for our models.
- **os**: To get the current working directory and join paths.

## Functions

### `mobilnet_v2(img_dir, n=3)`

#### Parameters:
- **img_dir**: Directory where the image is located.
- **n**: The number of top classifications to return (default is 3).

#### Description:
This function classifies an image using MobileNetV2 model provided by ImageAI. It takes an image directory as an input and returns the top `n` class labels along with their associated probabilities.

### `simple_CNN(img_dir, n=3)`

#### Parameters:
- **img_dir**: Directory where the image is located.
- **n**: The number of top classifications to return (default is 3).

#### Description:
This function uses a simple CNN model implemented using PyTorch. The CNN architecture consists of a single convolutional layer followed by a ReLU activation and max-pooling. Finally, a fully connected layer is used for classification. The function returns the top `n` class labels with probabilities.

### `resnet_50(img_dir, n=3)`

#### Parameters:
- **img_dir**: Directory where the image is located.
- **n**: The number of top classifications to return (default is 3).

#### Description:
This function utilizes a pre-trained ResNet-50 model implemented in TensorFlow. The function returns the top `n` class labels with their probabilities. The ResNet-50 model is a 50-layer deep network trained for various image classification tasks.

## Files Required:
1. **mobilenet_v2-b0353104.pth**: Pre-trained MobileNetV2 model weights.
2. **cnn_model_trained.pth**: Pre-trained weights for Simple CNN.
3. **resnet_model_tf.h5**: Pre-trained ResNet-50 model.
4. **names_dict.json**: JSON file that maps class indices to human-readable names.

In [1]:
img_dir = "./animal_images/lion.jpg"
n = 5

In [2]:
from PIL import Image
from imageai.Classification import ImageClassification
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import json
import os


def mobilnet_v2(img_dir, n=3):
    exec_path = os.getcwd()

    prediction = ImageClassification()
    prediction.setModelTypeAsMobileNetV2()
    prediction.setModelPath(os.path.join(exec_path, "mobilenet_v2-b0353104.pth"))
    prediction.loadModel()

    predictions, probabilities = prediction.classifyImage(
        os.path.join(exec_path, img_dir), result_count=n
    )

    results = []
    for eachPred, eachProb in zip(predictions, probabilities):
        class_label = eachPred.capitalize()
        probability = eachProb  # Convert probability to percentage
        results.append(f"{class_label}: {probability:.1f}%")

    return ", ".join(results)


def simple_CNN(img_dir, n=3):
    with open("./animal_name_translation/names_dict.json") as f:
        class_translation = json.load(f)

    class SimpleCNN(nn.Module):
        def __init__(self, num_classes):
            super(SimpleCNN, self).__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
            )
            self.classifier = nn.Linear(16 * 112 * 112, num_classes)

        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), -1)
            x = self.classifier(x)
            return x

    model = SimpleCNN(num_classes=len(class_translation))
    model.load_state_dict(torch.load("./cnn_model_trained.pth"))
    model.eval()

    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]
    )
    input_image = Image.open(img_dir)
    input_tensor = transform(input_image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)

    probabilities = nn.functional.softmax(output[0], dim=0)
    top_probabilities, top_class_indices = torch.topk(probabilities, k=n)

    results = []
    for prob, class_idx in zip(top_probabilities, top_class_indices):
        class_label = class_translation.get(str(class_idx.item()), "Unknown")
        probability = prob * 100  # Convert probability to percentage
        results.append(f"{class_label}: {probability:.1f}%")

    return ", ".join(results)


def resnet_50(img_dir, n=3):
    # Load the translation JSON file
    with open("./animal_name_translation/names_dict.json") as f:
        class_translation = json.load(f)

    # Load the trained model
    model = tf.keras.models.load_model("./resnet_model_tf.h5")

    # Load and preprocess the image for prediction
    img_path = img_dir  # Replace with the path to your image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image data

    predictions = model(img_array)

    # Get the indices that would sort the array in descending order
    sorted_indices = np.argsort(predictions, axis=1)[:, ::-1]
    # Get the top n indices
    top_n_indices = sorted_indices[:, :n]
    
    results = []
    for i in top_n_indices[0]:
        class_label = class_translation.get(str(i), "Unknown")
        probability = predictions[0][i] * 100  # Convert probability to percentage
        results.append(f"{class_label}: {probability:.1f}%")
    
    return ", ".join(results)

C:\Users\iziok\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\iziok\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\iziok\anaconda\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [3]:
result_mnv2 = mobilnet_v2(img_dir, n)
result_cnn = simple_CNN(img_dir, n)
result_rn50 = resnet_50(img_dir, n)

In [4]:
print(result_mnv2)
print(result_cnn)
print(result_rn50)

Lion: 99.8%, Collie: 0.1%, Shetland sheepdog: 0.0%, Cheetah: 0.0%, Timber wolf: 0.0%
Lion: 99.2%, Cheetah: 0.3%, Corn Snake: 0.2%, Red Fox: 0.1%, Golden Poison Dart Frog: 0.0%
Lion: 37.3%, American Robin: 28.6%, Grizzly Bear: 12.8%, Cheetah: 6.7%, Komodo Dragon: 3.5%
